# Predictions using the location informations and the prices.

## Load CSV files

In [26]:
import pandas as pd

# Load the two CSV files into dataframes
merge_df = pd.read_csv('\Users\Sysadmin\Downloads\Coworking-main\Coworking-main\merged_df.csv')

# Display the first few rows to confirm the data is loaded correctly
print(merge_df.head())


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-1: truncated \UXXXXXXXX escape (3754768711.py, line 4)

In [ ]:
# Check the minimum and maximum values of each column
print(merged_df[['Population', 'Household average net rent']].min())
print(merged_df[['Population', 'Household average net rent']].max())

# Check for any non-numeric entries
print(merged_df[['Population', 'Household average net rent']].dtypes)


Population                    10816.0
Household average net rent    40008.0
dtype: float64
Population                    10816.0
Household average net rent    40008.0
dtype: float64
Population                    float64
Household average net rent      int64
dtype: object


## Merge the dataframes.

In [2]:
# Check the columns of both dataframes
print(coworking_info_df.columns)
print(prices_df.columns)


Index(['Unnamed: 0', 'Neightboorhood', 'Population',
       'Percentage of population between 16 and 64 years', 'ForeignPopulation',
       'Unemployes', 'Household average net rent', 'Metro', 'Closest bus stop',
       'formatted_address', 'lat', 'lng'],
      dtype='object')
Index(['Unnamed: 0', 'EslabOn', 'Area', 'Freeland', 'Talent Garden', 'Loom'], dtype='object')


In [3]:
# Rename the 'Unnamed: 0' column to 'Coworking Area' in coworking_info_df
coworking_info_df.rename(columns={'Unnamed: 0': 'Coworking Area'}, inplace=True)

In [4]:
# Transpose the prices dataframe to make the coworking areas as rows
prices_df_transposed = prices_df.transpose()

# Reset index and rename the columns appropriately
prices_df_transposed.reset_index(inplace=True)
prices_df_transposed.columns = ['Coworking Area', 'Mesa Fija', 'Mesa Flexible', 'Despacho Priado', 'Pases/Bonos (días)']


In [ ]:
# Merge the two dataframes on 'Coworking Area'
merged_df = pd.merge(coworking_info_df, prices_df_transposed, how='inner', on='Coworking Area')

# Display the merged dataframe
print(merged_df.head())


  Coworking Area Neightboorhood  Population  \
0       Freeland         Cortes     10816.0   

  Percentage of population between 16 and 64 years ForeignPopulation  \
0                                            76,4%             28,4%   

   Unemployes  Household average net rent  Metro Closest bus stop  \
0         438                       40008     10             110m   

                                   formatted_address        lat       lng  \
0  C. de Gandía, 1, local 9A, Retiro, 28007 Madri...  40.398585 -3.671745   

  Mesa Fija Mesa Flexible Despacho Priado Pases/Bonos (días)  
0       150            95             375                 20  


## Data Processing.

In [11]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 16 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Coworking Area                                    1 non-null      object 
 1   Neightboorhood                                    1 non-null      object 
 2   Population                                        1 non-null      float64
 3   Percentage of population between 16 and 64 years  1 non-null      float64
 4   ForeignPopulation                                 1 non-null      float64
 5   Unemployes                                        1 non-null      int64  
 6   Household average net rent                        1 non-null      int64  
 7   Metro                                             1 non-null      int64  
 8   Closest bus stop                                  1 non-null      object 
 9   formatted_address        

In [15]:
import pandas as pd

# Convert price columns to numeric values
merged_df['Mesa Fija'] = pd.to_numeric(merged_df['Mesa Fija'], errors='coerce')
merged_df['Mesa Flexible'] = pd.to_numeric(merged_df['Mesa Flexible'], errors='coerce')
merged_df['Despacho Priado'] = pd.to_numeric(merged_df['Despacho Priado'], errors='coerce')
merged_df['Pases/Bonos (días)'] = pd.to_numeric(merged_df['Pases/Bonos (días)'], errors='coerce')

# Check the dataframe after cleaning
print(merged_df.head())


  Coworking Area Neightboorhood  Population  \
0       Freeland         Cortes     10816.0   

   Percentage of population between 16 and 64 years  ForeignPopulation  \
0                                             0.764              0.284   

   Unemployes  Household average net rent  Metro Closest bus stop  \
0         438                       40008     10             110m   

                                   formatted_address        lat       lng  \
0  C. de Gandía, 1, local 9A, Retiro, 28007 Madri...  40.398585 -3.671745   

   Mesa Fija  Mesa Flexible  Despacho Priado  Pases/Bonos (días)  
0        150             95              375                  20  


In [7]:
# Clean the percentage columns by replacing commas with dots and removing the '%' symbol
merged_df['Percentage of population between 16 and 64 years'] = merged_df['Percentage of population between 16 and 64 years'] \
    .str.replace(',', '.', regex=False) \
    .str.replace('%', '', regex=False) \
    .astype(float) / 100

# Clean the 'ForeignPopulation' column (if necessary)
merged_df['ForeignPopulation'] = merged_df['ForeignPopulation'] \
    .str.replace(',', '.', regex=False) \
    .str.replace('%', '', regex=False) \
    .astype(float) / 100


In [8]:
print(merged_df[['Percentage of population between 16 and 64 years', 'ForeignPopulation']].head())

   Percentage of population between 16 and 64 years  ForeignPopulation
0                                             0.764              0.284


In [17]:
from sklearn.preprocessing import MinMaxScaler

# Select the columns to normalize (numerical columns)
columns_to_normalize = ['Population', 'Household average net rent', 'Mesa Fija', 'Mesa Flexible', 'Despacho Priado', 'Pases/Bonos (días)']

# Apply MinMaxScaler to normalize data
scaler = MinMaxScaler()

merged_df[columns_to_normalize] = scaler.fit_transform(merged_df[columns_to_normalize])

print(merged_df[columns_to_normalize].head())


   Population  Household average net rent  Mesa Fija  Mesa Flexible  \
0         0.0                         0.0        0.0            0.0   

   Despacho Priado  Pases/Bonos (días)  
0              0.0                 0.0  


## Visualizations.

### Heatmap

### Radar Chart.

### Bar chart